In [1]:
%env DWAVE_API_TOKEN= 
import torch
from dwave.system import DWaveSampler

env: DWAVE_API_TOKEN=


In [2]:
def float_to_binary_list(num: float) -> list:
    # 
    rounded_num = round(num)
    
    # 
    if rounded_num < 0:
        raise ValueError("")
    
    # '0b'
    binary_str = bin(rounded_num)[2:]
    
    # 
    if len(binary_str) > 8:
        raise ValueError("")
    
    # 
    binary_list = [int(bit) for bit in binary_str.zfill(8)]
    
    return binary_list

In [3]:
h = torch.load('h.pt')
J = torch.load('J.pt')

In [41]:
import numpy as np

In [50]:
condition_energy_list = np.random.randint(0, 255, size=100)

In [51]:
print(condition_energy_list)

[ 22 142 145 232  97 164  51  65   6 194 159 180  10  13 226   7  24 222
 250  51 171 225 178 197  51 143 243  63  63  80 169 119   2  25 123  59
  27  86  41 232 241 119  46 218 168 178 132 177 105 254 247 240  42  10
 237 211 233 131  28   6 114  51  27 232  92 147 209 218  29 229  22 197
 100 246  37 212  90 190 227  79  63  47  71  98 150 146 234 182  28 136
 178 227  11   7 105 121  13 131 188 196]


In [52]:
conditioned_sample_energy_list = []
for energy in condition_energy_list:
    condition_energy_binary = float_to_binary_list(energy)
    encoded_nodes_list = [3120, 3165, 3060, 3105, 3000, 3045, 3300, 3345]
    for i, node in enumerate(encoded_nodes_list):
        h[node] = -8.0* condition_energy_binary[i] + 4.0

    qpu_sampler = DWaveSampler(solver={'topology__type': 'pegasus',  "chip_id":"Advantage_system6.4"})
    response = qpu_sampler.sample_ising(h, J, num_reads=1, auto_scale=False)
    batch_samples, batch_energies = response.record['sample'], response.record['energy']
    qubit_idxs = response.variables

    condition_state_samples = []
    for state in batch_samples:
        for node in encoded_nodes_list:
            where = torch.where(torch.tensor(qubit_idxs) == node)[0][0].item()
            condition_state_samples.append(int((state[where]+1)/2))

    binary_str = ''.join(str(bit) for bit in condition_state_samples)
    conditioned_sample_energy = int(binary_str, 2)
    conditioned_sample_energy_list.append(conditioned_sample_energy)

In [53]:
print(conditioned_sample_energy_list)

[22, 142, 145, 232, 97, 164, 51, 65, 6, 194, 159, 180, 10, 13, 226, 7, 24, 222, 250, 51, 171, 225, 178, 197, 51, 143, 243, 63, 63, 80, 169, 119, 2, 25, 123, 59, 27, 86, 41, 232, 241, 119, 46, 218, 168, 178, 132, 177, 105, 254, 247, 240, 42, 10, 237, 211, 233, 131, 28, 6, 114, 51, 27, 232, 92, 147, 209, 218, 29, 229, 22, 197, 100, 246, 37, 212, 90, 190, 227, 79, 63, 47, 71, 98, 150, 146, 234, 182, 28, 136, 178, 227, 11, 7, 105, 121, 13, 131, 188, 196]


In [85]:
if np.array_equal(condition_energy_list, conditioned_sample_energy_list):
    print("Success!")


Success!


In [70]:

condition_energy_binary = float_to_binary_list(139)
encoded_nodes_list = [3120, 3165, 3060, 3105, 3000, 3045, 3300, 3345]
for i, node in enumerate(encoded_nodes_list):
    h[node] = -8.0* condition_energy_binary[i] + 4.0

qpu_sampler = DWaveSampler(solver={'topology__type': 'pegasus',  "chip_id":"Advantage_system6.4"})
response = qpu_sampler.sample_ising(h, J, num_reads=100, auto_scale=False)
batch_samples, batch_energies = response.record['sample'], response.record['energy']
qubit_idxs = response.variables


In [75]:
wrong_number = 0
for state in batch_samples:
    condition_state_samples = []
    for node in encoded_nodes_list:
        where = torch.where(torch.tensor(qubit_idxs) == node)[0][0].item()
        condition_state_samples.append(int((state[where]+1)/2))
    binary_str = ''.join(str(bit) for bit in condition_state_samples)
    conditioned_sample_energy = int(binary_str, 2)
    if conditioned_sample_energy != 139:
        wrong_number += 1
print('The number of unmatched conditioned energies in 100 samples:', wrong_number)

The number of unmatched conditioned energies in 100 samples: 0
